# Libraries

In [ ]:
import numpy as np 
import pandas as pd 
from sklearn import preprocessing
import tensorflow as tf
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import Normalizer
import os
from tensorflow.keras import layers
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Flatten,Embedding,Dense
from keras.layers.merge import Concatenate

import keras

import warnings
warnings.filterwarnings('ignore')



# Preprocessing

In [ ]:
train = pd.read_csv("../input/store-sales-time-series-forecasting/train.csv")

test = pd.read_csv("../input/store-sales-time-series-forecasting/test.csv")

oil = pd.read_csv("../input/store-sales-time-series-forecasting/oil.csv")


def split_seq(a,split_length):
    x=np.zeros((split_length,a.shape[1]))    
    x=np.vstack([x,a])
    sales_prev=[]
    sales_next=[]

    for i in range(split_length,x.shape[0]): 
        sales_prev.append(x[i-split_length:i])
        sales_next.append(x[i])

    
    return np.array((sales_prev)),np.array((sales_next))

split_len=16    
    
pivoted_train = train.pivot(index=['date'], columns=['store_nbr', 'family'], values=['sales'])


pivoted_train_clone=pivoted_train


pivoted_train_clone=pivoted_train_clone.merge(oil, on=['date'], how="left")

pivoted_train_clone['dcoilwtico']=pivoted_train_clone['dcoilwtico'].fillna(method='bfill')

oil=pivoted_train_clone['dcoilwtico']
del pivoted_train_clone
display(oil)


display(pivoted_train)

scaler3 = MinMaxScaler(feature_range=(-1, 1))

pivoted_train = scaler3.fit_transform(pivoted_train)


pivoted_train,pivoted_train_y=split_seq(pivoted_train,split_len)
oil=oil.values.reshape(-1,1)


scaler2 = MinMaxScaler(feature_range=(-1, 1))

oil = scaler2.fit_transform(oil.reshape(-1,1))

print(oil.shape)
print(pivoted_train.shape)



# Model and Train

In [ ]:
def network():
    inp = layers.Input(shape=(split_len, 1782,))

    x=layers.MultiHeadAttention(
        key_dim=256, num_heads=4, dropout=0.25
    )(inp, inp)
    
    x=keras.layers.Dropout(0.25)(x)

    x = layers.LayerNormalization(epsilon=1e-6)(x)

    res = x + inp
    # Feed Forward Part
    
    x = layers.Conv1D(filters=4, kernel_size=1, activation="relu")(res)
    x = layers.Dropout(0.25)(x)
    x = layers.Conv1D(filters=1782, kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    x=x+res
    
    
    
    
    
    x=layers.MultiHeadAttention(
        key_dim=256, num_heads=4, dropout=0.25
    )(x, x)
    
    x=keras.layers.Dropout(0.25)(x)

    x = layers.LayerNormalization(epsilon=1e-6)(x)

    res = x + inp
    # Feed Forward Part
    
    x = layers.Conv1D(filters=4, kernel_size=1, activation="relu")(res)
    x = layers.Dropout(0.25)(x)
    x = layers.Conv1D(filters=1782, kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    x=x+res
    
    
    
    
    
    
    x=layers.MultiHeadAttention(
        key_dim=256, num_heads=4, dropout=0.25
    )(x, x)
    
    x=keras.layers.Dropout(0.25)(x)

    x = layers.LayerNormalization(epsilon=1e-6)(x)

    res = x + inp
    # Feed Forward Part
    
    x = layers.Conv1D(filters=4, kernel_size=1, activation="relu")(res)
    x = layers.Dropout(0.25)(x)
    x = layers.Conv1D(filters=1782, kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    x=x+res
    
    
    
    
    
    x=layers.MultiHeadAttention(
        key_dim=256, num_heads=4, dropout=0.25
    )(x, x)
    
    x=keras.layers.Dropout(0.25)(x)

    x = layers.LayerNormalization(epsilon=1e-6)(x)

    res = x + inp
    # Feed Forward Part
    
    x = layers.Conv1D(filters=4, kernel_size=1, activation="relu")(res)
    x = layers.Dropout(0.25)(x)
    x = layers.Conv1D(filters=1782, kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    x=x+res
    
    
    
    
    
    
    
    
    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    
    x=keras.layers.Dense(512,activation="relu")(x)
    
    x=keras.layers.Dropout(0.4)(x)
    
    x=keras.layers.Dense(512,activation="relu")(x)
    
    x=keras.layers.Dropout(0.4)(x)
    

    out=keras.layers.Dense(1782)(x)
    
    model = keras.Model(inputs=[inp], outputs=out)
    return model

model=network()
print(model.summary())
model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam())



model.fit(x=[pivoted_train],y=pivoted_train_y,batch_size=256,epochs=400,validation_split=0.05)

# Test

In [ ]:
test = pd.read_csv("../input/store-sales-time-series-forecasting/test.csv")
id=test['id']

display(test)
test=pivoted_train_y[pivoted_train_y.shape[0]-split_len:]
test=np.expand_dims(test,axis=0)

sales=np.array([])
for i in range(16):
    predictions = model.predict(test)
    predictions=predictions.reshape(-1)
    #sales+=predictions.reshape(-1)
    test=np.insert(test,-1,predictions,axis=1)
    test=np.delete(test, 0, axis=1)
    sales=np.append(sales,predictions)
    #test = np.array([test,predictions])
    #predictions=np.expand_dims(predictions,axis=0)
    #test=np.append(test[1:],predictions,axis=0)


y_predict = pd.DataFrame(scaler3.inverse_transform(sales.reshape((16, 1782))))
sales=y_predict.values.reshape(16*1782)
sales = sales.clip(min=0)
test_results=pd.DataFrame({'id':id,'sales':sales})
display(test_results)
test_results.to_csv('results.csv' , index=False)